In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as tf_image
from sklearn.metrics import classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [6]:
# ==============================================================================
# Step 1: Configuration & Environment Setup
# ==============================================================================

# Define the path to your dataset folder.
# The folder should contain subfolders for each class (e.g., 'Cassava___healthy').
DATA_DIR = './dataset'

# Define image dimensions and batch size.
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Path for a user-provided image to classify.
# This variable will be updated later with user input.
user_image_path = ''

# Check if the dataset directory exists.
if not os.path.exists(DATA_DIR):
    print(f"Error: Dataset directory '{DATA_DIR}' not found.")
    print("Please make sure you have the 'dataset' folder with the subfolders "
          "('Cassava___bacterial_blight', etc.) in the same directory as this script.")
    exit()

In [7]:
# ==============================================================================
# Step 2: Data Loading and Pre-processing
# ==============================================================================
print("Loading and pre-processing data...")

# Use ImageDataGenerator for data augmentation and pre-processing.
# Rescale pixel values from [0, 255] to [0, 1].
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of data for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data.
train_ds = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data.
val_ds = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get the class names from the dataset.
class_names = list(train_ds.class_indices.keys())
print(f"Found {len(class_names)} classes: {class_names}")

Loading and pre-processing data...
Found 17120 images belonging to 5 classes.
Found 4277 images belonging to 5 classes.
Found 5 classes: ['Cassava___bacterial_blight', 'Cassava___brown_streak_disease', 'Cassava___green_mottle', 'Cassava___healthy', 'Cassava___mosaic_disease']


In [8]:
# ==============================================================================
# Step 3: Build the CNN Model
# ==============================================================================
print("Building the CNN model...")

model = models.Sequential([
    # Input layer and first convolutional block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),
    # Second convolutional block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Third convolutional block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Fourth convolutional block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the 3D output to 1D for the dense layers.
    layers.Flatten(),
    # Fully connected layers
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5), # Add dropout to prevent overfitting
    layers.Dense(len(class_names), activation='softmax') # Output layer
])

# Compile the model with an optimizer, loss function, and metrics.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Building the CNN model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin 